In [17]:
import requests
import json
import PyPDF2
import os

In [ ]:
class SimpleLlamaQA:
    def __init__(self):
        self.documents = ""
        self.ollama_url = "http://localhost:11434"
    
    # Step 3: Load PDF files
    def load_pdf(self, file_path):
        text = ""
        with open(file_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            for page in pdf_reader.pages:
                text += page.extract_text() + "\n"
        return text
    
    # Step 4: Load TXT files
    def load_txt(self, file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    
    # Step 5: Clean and extract important text
    def clean_text(self, text):
        # Remove extra whitespace and empty lines
        lines = [line.strip() for line in text.split('\n') if line.strip()]
        clean_text = '\n'.join(lines)
        
        # Keep only first 4000 characters to fit in context
        if len(clean_text) > 4000:
            clean_text = clean_text[:4000] + "..."
        
        return clean_text
    
    # Step 6: Upload multiple files
    def upload_files(self, file_paths):
        all_text = ""
        
        for file_path in file_paths:
            print(f"Loading: {file_path}")
            
            if file_path.endswith('.pdf'):
                text = self.load_pdf(file_path)
            elif file_path.endswith('.txt'):
                text = self.load_txt(file_path)
            else:
                print(f"Unsupported file type: {file_path}")
                continue
            
            # Clean the text
            clean_text = self.clean_text(text)
            all_text += f"\n\n=== {os.path.basename(file_path)} ===\n{clean_text}"
        
        self.documents = all_text
        print(f"✅ Loaded {len(file_paths)} files")
        return True
    
     # Step 7: Ask questions to Llama 3
    def ask_question(self, question):
        if not self.documents:
            return "No documents loaded!"
        
        # Create simple prompt
        prompt = f"""Based on the following documents, answer this question: {question}

Documents:
{self.documents}

Answer based only on the information in the documents:"""
        
        # Send to Llama 3
        try:
            # First check what models are available
            models_response = requests.get(f"{self.ollama_url}/api/tags")
            if models_response.status_code == 200:
                available_models = [m['name'] for m in models_response.json()['models']]
                print(f"Available models: {available_models}")
                
                # Try to find llama3 variant
                model_name = "llama3"
                if "llama3" not in available_models:
                    for model in available_models:
                        if "llama" in model.lower():
                            model_name = model
                            print(f"Using model: {model_name}")
                            break
                    else:
                        return "❌ No Llama model found. Run: ollama pull llama3"
            
            response = requests.post(
                f"{self.ollama_url}/api/generate",
                json={
                    "model": model_name,
                    "prompt": prompt,
                    "stream": False
                }
            )
            
            if response.status_code == 200:
                return response.json()['response']
            else:
                return f"Error: {response.status_code}"
        
        except Exception as e:
            return f"Error: {e}"


In [28]:
def main():
    # Create the system
    qa = SimpleLlamaQA()
    
    # Example files (replace with your file paths)
    files = [
        "test.txt"
    ]
    
    # Upload files
    qa.upload_files(files)
    
    # Ask questions
    questions = [
        "What is the name of the person?",
        "What does this person have?"
    ]
    
    for question in questions:
        print(f"\nQ: {question}")
        answer = qa.ask_question(question)
        print(f"A: {answer}")
        print("-" * 50)

In [36]:
if __name__ == "__main__":
    # Create QA system
    main()

Loading: test.txt
✅ Loaded 1 files

Q: What is the name of the person?
Available models: ['llama3:latest']
Using model: llama3:latest
A: According to the document "test.txt", the name of the person is Mike.
--------------------------------------------------

Q: What does this person have?
Available models: ['llama3:latest']
Using model: llama3:latest
A: According to the document "test.txt", Mike has a big brain.
--------------------------------------------------
